In [188]:
'''

USGS (US Geological Survey) publishes earthquake data. Here’s a feed spanning all domestic earthquakes from the past month:
https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.geojson

Using this JSON feed:
1) identify every earthquake in California from past month,
2) list them by increasing magnitude
3. and finally output in a format resembling the following e.g.:

2017-07-13T22:32:48+00:00 | 15km SE of Mammoth Lakes, California | Magnitude: 0.92
2017-07-13T22:49:58+00:00 | 8km ENE of Mammoth Lakes, California | Magnitude: 0.92
2017-07-13T22:37:52+00:00 | 12km E of Mammoth Lakes, California | Magnitude: 0.95
2017-07-13T20:43:37+00:00 | 3km NW of Greenville, California | Magnitude: 1
2017-07-13T22:31:04+00:00 | 11km E of Mammoth Lakes, California | Magnitude: 1.31
2017-07-13T22:45:28+00:00 | 37km SE of Bridgeport, California | Magnitude: 1.7
2017-07-13T22:54:30+00:00 | 3km SE of Atascadero, California | Magnitude: 2.04
2017-07-13T22:09:53+00:00 | 41km SW of Ferndale, California | Magnitude: 2.76

'''

In [1]:
import json
import requests
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
filename = './all_month.geojson'
url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.geojson'

def load_data():
    try:
        resp = requests.get(url)
        resp_json = resp.json()
        
        with open(filename, 'w') as f:
            json.dump(resp_json, f)
        return resp_json
    
    except Exception:
        with open(filename, 'r') as f:
            return json.load(f)

data = load_data()

In [5]:
# store as DataFrame
df = pd.io.json.json_normalize(data['features'])
data_size = len(df)
properties = list(df)
sources = df['properties.type'].unique()

import time

start = time.time()

df = df[['properties.time', 'properties.place', 'properties.mag']] # keep columns relevant to output
# df = df[['properties.time', 'properties.place', 'properties.mag', 'properties.type']] # keep columns relevant to output
df_cali = df[df['properties.place'].str.contains('California|CA')] # find entries with 'place' value containing 'California' or 'CA', 628 records
# df_cali = df[(df['properties.place'].str.contains('California|CA')) & (df['properties.type'] == 'earthquake')] # find entries with 'place' value containing 'California' or 'CA'
df_cali_sorted = df_cali.sort_values(['properties.mag', 'properties.time']) # sort by magnitude, then time occurred
# df_cali_sorted['properties.time'] = pd.to_datetime(df_cali_sorted['properties.time'], unit='ms').dt.tz_localize('UTC').dt.strftime('%Y-%m-%dT%H:%M:%S%z') # unit=milliseconds
# df_cali_sorted['properties.time'] = pd.to_datetime(df_cali_sorted['properties.time'], unit='ms').dt.tz_localize('UTC').dt.strftime('%Y-%m-%dT%H:%M:%S') # unit=milliseconds
# df_cali_sorted['properties.time'] = df_cali_sorted['properties.time'].apply(format_time)

results = []
print len(df_cali_sorted)


format_time = lambda x: datetime.utcfromtimestamp(int(x) / 1000.0).strftime('%Y-%m-%dT%H:%M:%S') + '+00:00'
format_place = lambda x: str(x)
format_mag = lambda x: 'Magnitude: {}'.format(x) # better to use str.format as it converts its arguments into strings
format_output = lambda (idx, row): '{} | {} | {}'.format(format_time(row['properties.time']), 
                                                format_place(row['properties.place']), 
                                                format_mag(row['properties.mag']))


results = map(format_output, df_cali_sorted.iterrows())


# time 0.67157292366, iterrows is slower


# output format: 2017-07-13T22:32:48+00:00 | 15km SE of Mammoth Lakes, California | Magnitude: 0.92

3267


In [4]:
import matplotlib.pyplot as plt

# store as GeoPandas DataFrame
gdf = gpd.GeoDataFrame(data)
data_size = len(gdf)
properties = list(gdf)
# print data_size
print list(gdf)
# df['type'].unique()
gdf = gdf[['time', 'place', 'mag', 'geometry']] # keep columns relevant to output
gdf_cali = gdf[gdf['place'].str.contains('California|CA')] # find entries with 'place' value containing 'California' or 'CA', 628 records
gdf_cali_sorted = gdf_cali.sort_values(['mag', 'time']) # sort by magnitude, then time occurred
# format_time = lambda x: datetime.utcfromtimestamp(x / 1000.0).strftime('%Y-%m-%dT%H:%M:%S') + '+00:00'
# gdf_cali_sorted['time'] = gdf_cali_sorted['time'].apply(format_time)

# results = []

print len(gdf_cali_sorted)

(fig, ax) = plt.subplots(figsize=(40,40))
# set aspect to equal. This is done automatically
# when using *geopandas* plot on it's own, but not when
# working with pyplot directly.
ax.set_aspect('equal')

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
usa = world[world['name'] == 'United States']
usa.plot(ax=ax, color='chartreuse', edgecolor='black')
gdf_cali_sorted.plot(ax=ax, column='mag', cmap='OrRd', markersize=5)

plt.show()
gdf_cali_sorted

format_time = lambda x: datetime.utcfromtimestamp(int(x) / 1000.0).strftime('%Y-%m-%dT%H:%M:%S') + '+00:00'
format_place = lambda x: str(x)
format_mag = lambda x: 'Magnitude: {}'.format(x) # better to use str.format as it converts its arguments into strings

for (index, row) in gdf_cali_sorted.iterrows():
#     record = str(row['time']) + ' | ' + str(row['place']) + ' | ' + 'Magnitude: ' + str(row['mag'])
    record = '{} | {} | {}'.format(format_time(row['time']), 
                                   format_place(row['place']), 
                                   format_mag(row['mag']))
    print record

# output format: 2017-07-13T22:32:48+00:00 | 15km SE of Mammoth Lakes, California | Magnitude: 0.92

ValueError: arrays must all be same length

In [2]:
from datetime import datetime

'''
# slower, time=0.00527501106262
for (i, entry) in enumerate(data['features']):
    place = entry['properties']['place']
#     cause = entry['properties']['type']
    if ('California' in place) or ('CA' in place):
#     if (('California' in place) or ('CA' in place)) and (cause == 'earthquake'):
        record = {}
        record['place'] = place
        record['time'] = entry['properties']['time']
        record['magnitude'] = entry['properties']['mag']
        cali.append(record)
        # check types above (do we still have to if we use lambda?)
'''
    
in_california = lambda x: ('California' in x['properties']['place']) or \
                          ('CA' in x['properties']['place'])
select_fields = lambda x: {'time': x['properties']['time'], 
                           'place': x['properties']['place'], 
                           'magnitude': x['properties']['mag']}

cali = filter(in_california, data['features'])
cali = map(select_fields, cali)
# faster, time = 0.00476717948914
        
print len(cali)
# print cali[0] # {'magnitude': 1.75, 'place': '5km NW of The Geysers, CA', 'time': 1524287209810}

format_time = lambda x: datetime.utcfromtimestamp(int(x) / 1000.0).strftime('%Y-%m-%dT%H:%M:%S') + '+00:00'
format_place = lambda x: str(x)
format_mag = lambda x: 'Magnitude: {}'.format(x) # better to use str.format as it converts its arguments into strings
format_output = lambda x: '{} | {} | {}'.format(format_time(x['time']), 
                                                format_place(x['place']), 
                                                format_mag(x['magnitude']))

cali_sorted = sorted(cali, key=lambda x: (x['magnitude'], x['time']))

results = map(format_output, cali_sorted)

for result in results:
    print result
    
# time 0.389930963516, faster

# output format: 2017-07-13T22:32:48+00:00 | 15km SE of Mammoth Lakes, California | Magnitude: 0.92
# time conversion function
# check data type?

NameError: name 'data' is not defined

In [47]:
import pandas as pd
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot
from plotly.graph_objs import *

init_notebook_mode(connected=True)
mapbox_access_token = 'pk.eyJ1IjoieW91Z29lcmljIiwiYSI6ImNqZ2R4cmpsbTI3dzcycXFzdHRrdnFobGUifQ.xwe3ccbSH-QrCY9WPoYJ2w'

df = pd.io.json.json_normalize(data['features'])
df = df[['properties.time', 'properties.place', 'properties.mag', 'geometry.coordinates']] # keep columns relevant to output
df_cali = df[df['properties.place'].str.contains('California|CA')] # find entries with 'place' value containing 'California' or 'CA', 628 records
df_cali = df_cali.rename(index=str, columns={'properties.time': 'time', 'properties.place': 'place', 'properties.mag': 'mag'})
df_cali['long'] = df_cali['geometry.coordinates'].apply(lambda x: x[0])
df_cali['lat'] = df_cali['geometry.coordinates'].apply(lambda x: x[1])
df_cali = df_cali.drop(['geometry.coordinates'], axis=1)

site_lat = df_cali.lat
site_lon = df_cali.long
locations_name = df_cali.place

format_time = lambda x: datetime.utcfromtimestamp(int(x) / 1000.0).strftime('%Y-%m-%dT%H:%M:%S') + '+00:00'

df_cali_sorted = df_cali.sort_values(['mag', 'time'], ascending=True) # sort by magnitude, then time occurred
print df_cali_sorted.head()

scl = [ [0, "rgb(255, 160, 30)"], [1, "rgb(200, 0, 0)"] ]

data_cali = Data([
            Scattermapbox(
                lat = site_lat,
                lon = site_lon,
                mode = 'markers',
                marker = Marker(
                        size = 14,
                        color = df_cali['mag'],
                        colorscale = scl,
                        cmin = df_cali['mag'].min(),
                        cmax = df_cali['mag'].max(),
                        opacity = 0.8,
                        colorbar=dict(
                            title="Magnitude"
                            )
                        ),
                ),
            Scattermapbox(
                lat = site_lat,
                lon = site_lon,
                mode = 'markers',
                marker = Marker(
                    size = 6,
                    color = 'rgb(240, 200, 170)',
                    opacity = 0.5
                ),
            )]
        )
        
layout = Layout(
        title = 'Earthquake in California from Past Month',
        autosize = True,
        hovermode = 'closest',
        showlegend = False,
        mapbox = dict(
                accesstoken = mapbox_access_token,
                bearing = 0,
                center = dict(
                        lat=38,
                        lon=-98
                ),
        pitch = 0,
        zoom = 3,
        style = 'outdoors'
        ),
    )

fig = dict(data=data_cali, layout=layout)
iplot(fig, filename='Earthquake in California from Past Month')
plot(fig, filename='caliDataPlotly.html')

               time                               place   mag        long  \
9133  1522217374350                 6km WNW of Cobb, CA -0.41 -122.792667   
8698  1522327419335        3km E of Truckee, California -0.30 -120.141000   
8188  1522455888154      4km NNE of Truckee, California -0.30 -120.162800   
2048  1524030592481  2km WSW of Tahoe Vista, California -0.30 -120.081200   
2915  1523780879710                 8km WSW of Anza, CA -0.28 -116.747667   

            lat  
9133  38.835000  
8698  39.325600  
8188  39.364000  
2048  39.234200  
2915  33.518833  


'file:///Users/eric/Desktop/appl/caliDataPlotly.html'

In [14]:

in_california = lambda x: ('California' in x['properties']['place']) or \
                          ('CA' in x['properties']['place'])

cali = filter(in_california, data['features'])

for entry in cali:
    del entry['properties']['rms']
    del entry['properties']['code']
    del entry['properties']['cdi']
    del entry['properties']['sources']
    del entry['properties']['nst']
    del entry['properties']['tz']
    del entry['properties']['title']
    del entry['properties']['magType']
    del entry['properties']['detail']
    del entry['properties']['sig']
    del entry['properties']['net']
    del entry['properties']['status']
    del entry['properties']['updated']
    del entry['properties']['felt']
    del entry['properties']['alert']
    del entry['properties']['dmin']
    del entry['properties']['gap']
    del entry['properties']['types']
    del entry['properties']['url']
    del entry['properties']['ids']
    del entry['properties']['tsunami']
    del entry['properties']['mmi']

with open('cali.json', 'w') as f:
    json.dump(cali, f)

# faster, time = 0.00476717948914
        
print len(cali)
print cali[0]


3221
{u'geometry': {u'type': u'Point', u'coordinates': [-122.0886688, 39.4000015, 4.52]}, u'type': u'Feature', u'properties': {u'type': u'earthquake', u'mag': 1.78, u'place': u'16km NNE of Maxwell, CA', u'time': 1524667182790}, u'id': u'nc73005951'}
